## Importing the required Modules

In [2]:
!pip install transformers peft datasets torch bitsandbytes accelerate wandb datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!pip install datasets

## Cleaning the dataset

In [ ]:
import json

def clean_jsonl(input_file, output_file):
    cleaned_data = []
    with open(input_file, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            try:
                data = json.loads(line)
                if isinstance(data['response'], dict):
                    data['response'] = json.dumps(data['response'])
                cleaned_data.append(data)
            except json.JSONDecodeError:
                print(f"Error in line {line_num}")
                continue

    with open(output_file, 'w', encoding='utf-8') as f:
        for item in cleaned_data:
            f.write(json.dumps(item) + '\n')


#clean_jsonl('finetune_data1.jsonl', 'cleaned_finetune_data.jsonl')

## Fine-tuning Llama 3.2-1B for Robotics

This code fine-tunes a pre-trained Llama 3.2-1B language model for a robotics-related task. It utilizes:

* **LoRA (Low-Rank Adaptation):**  Efficiently fine-tunes the model by adding small, trainable matrices to its layers, reducing memory requirements.
* **4-bit Quantization:** Compresses the model's size and speeds up inference by representing its weights using fewer bits.
* **Hugging Face Transformers and Datasets:** Provides tools for loading, preprocessing, and training the model on a robotics dataset.
* **Weights & Biases (wandb):** Tracks the training progress and logs important metrics.

**Process:**

1. **Imports necessary libraries:** Including `transformers`, `peft`, `datasets`, and `wandb`.
2. **Defines a preprocessing function:** Formats the data for the model by creating instruction-response pairs.
3. **Defines the training function:**
    * Loads the pre-trained Llama 3.2-1B model and tokenizer.
    * Applies 4-bit quantization for memory efficiency.
    * Loads and preprocesses the robotics dataset.
    * Configures LoRA for fine-tuning.
    * Sets up training parameters using `TrainingArguments`.
    * Creates a `Trainer` instance to manage the training process.
    * Fine-tunes the model and saves the results.

In [5]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
import logging
import wandb
from huggingface_hub import login

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def preprocess_function(examples):
    texts = [
        f"### Instruction: {instruction}\n### Response: {response}"
        for instruction, response in zip(examples["instruction"], examples["response"])
    ]

    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )

    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

def train():
    wandb.init(project="llm-robotics-finetuning")

    global tokenizer
    model_id = "NousResearch/Llama-3.2-1B"

    # Configure 4-bit quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )

    # Load tokenizer with padding token
    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        padding_side="right",
        model_max_length=512
    )
    tokenizer.pad_token = tokenizer.eos_token

    # Load model with quantization
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.float16
    )

    # Load and preprocess dataset
    dataset = load_dataset("json", data_files="cleaned_finetune_data.jsonl")
    tokenized_dataset = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=dataset["train"].column_names
    )

    # Configure LoRA
    lora_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
    )

    # Prepare model
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)

    # Training arguments
    training_args = TrainingArguments(
        output_dir="./robotics_mistral",
        num_train_epochs=3,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        max_steps=500,
        optim="paged_adamw_32bit",
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
        save_strategy="steps",
        save_steps=100,
        evaluation_strategy="no",
        warmup_ratio=0.03
    )

    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        data_collator=data_collator
    )

    # Train and save
    trainer.train()
    trainer.save_model("./robotics_mistral_final")

if __name__ == "__main__":
    train()

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/630 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,2.177400
20,1.877600
30,1.601600
40,1.538700
50,1.395600
60,1.335500
70,1.275700
80,1.425600
90,1.185700
100,1.254600


In [ ]:
import torch
torch.cuda.empty_cache()

## Logging into Huggingface and creating a repo


In [ ]:
from huggingface_hub import create_repo, login
from google.colab import userdata

# Log in to Hugging Face Hub - Replace with your actual token
login(token=userdata.get('huggingface'))

repo_name = "SolomonMartin/robotics-llama-3.2-1b-finetuned"
create_repo(repo_name, private=False)  # Set private=False if you want it public

RepoUrl('https://huggingface.co/SolomonMartin/robotics-llama-3.2-1b-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='SolomonMartin/robotics-llama-3.2-1b-finetuned')

## Merging and Pushing Fine-tuned Llama Model

This code snippet focuses on merging the LoRA weights with the base Llama 3.2-1B model and pushing the fine-tuned model and tokenizer to the Hugging Face Hub.

**Process:**

1. **Imports necessary libraries:** Includes `peft` and `transformers`.
2. **Loads the base and LoRA models:**
    * Loads the pre-trained Llama 3.2-1B model.
    * Loads the LoRA weights from the fine-tuning step.
3. **Merges the models:**
    * Combines the base model and LoRA weights into a single, merged model.
    * Unloads the LoRA model to free up memory.
4. **Pushes to Hugging Face Hub:**
    * Uploads the merged model to the specified repository.
    * Uploads the tokenizer to the same repository.

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

# Load base model and LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-3.2-1B",
    torch_dtype=torch.float16,
    device_map="auto"
)
peft_model = PeftModel.from_pretrained(
    base_model,
    "./robotics_mistral_final"
)

# Merge weights (optional but recommended)
merged_model = peft_model.merge_and_unload()

# Push to Hub
merged_model.push_to_hub(
    repo_name,
    use_temp_dir=True,
    commit_message="Add fine-tuned robotics LLaMA model"
)

# Push tokenizer
tokenizer.push_to_hub(
    repo_name,
    use_temp_dir=True,
    commit_message="Add tokenizer"
)

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SolomonMartin/robotics-llama-3.2-1b-finetuned/commit/e50bd26da7fd2459d57eb53a01761222183e977a', commit_message='Add tokenizer', commit_description='', oid='e50bd26da7fd2459d57eb53a01761222183e977a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SolomonMartin/robotics-llama-3.2-1b-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='SolomonMartin/robotics-llama-3.2-1b-finetuned'), pr_revision=None, pr_num=None)